In [1]:
# Par exemple, au début de train_model.py :
import os
from pathlib import Path

# On peut forcer la base des données ici
# BASE_DIR = Path("/content/drive/MyDrive/PROJET_IA/...")
# mais normalement le parser argparse s’en charge.

# Vérifie que l’usage est bien :
# python scripts/train_model.py --data data/export_IA.csv --models-dir models scripts/train_model.py

In [3]:
!python train_model.py --csv After_Sort.csv --models-dir models --max-samples 50000


2025-06-12 23:28:14.776403: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-12 23:28:16.868178: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Traceback (most recent call last):
  File "c:\Users\Paul\Documents\GitHub\Projet_Outil_de_l-ing-_S6\IA\besoin4\train_model.py", line 144, in <module>
    main()
  File "c:\Users\Paul\Documents\GitHub\Projet_Outil_de_l-ing-_S6\IA\besoin4\train_model.py", line 126, in main
    tf.config.experimental.set_memory_growth(
  File "c:\Users\Paul\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\pyt

In [5]:
!python predict_position.py --input After_Sort.csv --models-dir models --output preds.csv


[INFO] Données chargées: 258102 lignes, 112 navires


2025-06-12 23:28:45.468068: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-12 23:28:47.259246: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Traceback (most recent call last):
  File "c:\Users\Paul\Documents\GitHub\Projet_Outil_de_l-ing-_S6\IA\besoin4\predict_position.py", line 210, in <module>
    main()
  File "c:\Users\Paul\Documents\GitHub\Projet_Outil_de_l-ing-_S6\IA\besoin4\predict_position.py", line 165, in main
    X_sequences, mmsis, timestamps = create_sequences_for_prediction(data, SEQUENCE_LENGTH)
                                     ^

In [ ]:
import pandas as pd

# Load the predictions from the CSV file
preds = pd.read_csv('preds.csv')

# Load the true data from the CSV file, specifying dtypes to handle mixed types
true_df = pd.read_csv('data/export_IA.csv', dtype={'VesselName': str, 'IMO': str, 'CallSign': str, 'VesselType': str, 'Status': str})


# 1. Créer une colonne "Date_Pred" dans preds = BaseDateTime + Horizon_min minutes
preds['BaseDateTime'] = pd.to_datetime(preds['BaseDateTime']) # Convert BaseDateTime to datetime
preds['Date_Pred'] = preds['BaseDateTime'] + pd.to_timedelta(preds['Horizon_min'], unit='m')

# Convert BaseDateTime in true_df to datetime
true_df['BaseDateTime'] = pd.to_datetime(true_df['BaseDateTime'])

# Renommer les colonnes LAT et LON pour les vraies données in true_df
true_df = true_df.rename(columns={"LAT": "True_LAT", "LON": "True_LON"})


# 2. Trier les dataframes par MMSI et date (nécessaire pour merge_asof dans chaque groupe)
preds = preds.sort_values(['MMSI', 'Date_Pred']).reset_index(drop=True)
true_df = true_df.sort_values(['MMSI', 'BaseDateTime']).reset_index(drop=True)

# Group by MMSI and perform merge_asof for each group
merged_list = []
for mmsi, preds_group in preds.groupby("MMSI"):
    true_group = true_df[true_df["MMSI"] == mmsi].copy()

    if not true_group.empty:
        merged_group = pd.merge_asof(
            preds_group,
            true_group[['BaseDateTime', 'True_LAT', 'True_LON']],
            left_on='Date_Pred',
            right_on='BaseDateTime',
            direction='nearest',  # nearest c’est mieux que 'backward' pour plus de précision
            tolerance=pd.Timedelta('5min')  # tolérance max, à ajuster si besoin
        )
        merged_list.append(merged_group)

# Concatenate the results
merged = pd.concat(merged_list, ignore_index=True)


# Display the head of the merged DataFrame to check the result
print("Head of the merged DataFrame:")
display(merged.head())

In [ ]:
import folium
from datetime import timedelta
from math import radians, sin, cos, sqrt, atan2

# Fonction haversine pour calculer la distance entre deux points GPS
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # rayon de la Terre en km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c  # distance en km

# Forcer le type datetime et trier
preds["BaseDateTime"] = pd.to_datetime(preds["BaseDateTime"])
true_df["BaseDateTime"] = pd.to_datetime(true_df["BaseDateTime"])
preds = preds.sort_values("BaseDateTime")
true_df = true_df.sort_values("BaseDateTime")

# Filtrer uniquement les prédictions à 5 min
preds_5min = preds[preds["Horizon_min"] == 5]

# Limiter à 10 bateaux
mmsi_list = preds_5min["MMSI"].unique()[:50]

# Calcul du centre de la carte
initial_points = []
for mmsi in mmsi_list:
    true_mmsi = true_df[true_df["MMSI"] == mmsi]
    preds_mmsi = preds_5min[preds_5min["MMSI"] == mmsi]
    if true_mmsi.empty or preds_mmsi.empty:
        continue
    initial_time = preds_mmsi.iloc[0]["BaseDateTime"]
    init_real_point = true_mmsi[true_mmsi["BaseDateTime"] == initial_time]
    if not init_real_point.empty:
        initial_points.append((
            init_real_point["True_LAT"].iloc[0],
            init_real_point["True_LON"].iloc[0]
        ))

if initial_points:
    avg_lat = sum([p[0] for p in initial_points]) / len(initial_points)
    avg_lon = sum([p[1] for p in initial_points]) / len(initial_points)
else:
    first_pred = preds_5min.iloc[0]
    avg_lat, avg_lon = first_pred["Predicted_LAT"], first_pred["Predicted_LON"]

# Créer la carte centrée
m = folium.Map(location=[avg_lat, avg_lon], zoom_start=8)

# Boucle sur chaque bateau limité à 10
for mmsi in mmsi_list:
    preds_mmsi = preds_5min[preds_5min["MMSI"] == mmsi].copy()
    true_mmsi = true_df[true_df["MMSI"] == mmsi].copy()

    if preds_mmsi.empty or true_mmsi.empty:
        continue

    row = preds_mmsi.iloc[0]
    initial_time = row["BaseDateTime"]
    pred_time = initial_time + timedelta(minutes=int(row["Horizon_min"]))

    pred_lat = row["Predicted_LAT"]
    pred_lon = row["Predicted_LON"]

    window = timedelta(minutes=2)
    nearby = true_mmsi[
        (true_mmsi["BaseDateTime"] >= pred_time - window) &
        (true_mmsi["BaseDateTime"] <= pred_time + window)
    ]

    if not nearby.empty:
        real_lat = nearby["True_LAT"].mean()
        real_lon = nearby["True_LON"].mean()
    else:
        real_lat = None
        real_lon = None

    initial_real_point = true_mmsi[true_mmsi["BaseDateTime"] == initial_time]

    # Marqueur point initial réel
    if not initial_real_point.empty:
        folium.Marker(
            location=[initial_real_point["True_LAT"].iloc[0], initial_real_point["True_LON"].iloc[0]],
            popup=f"Bateau {mmsi} - Initial Real",
            icon=folium.Icon(color="green", icon="ship", prefix='fa')
        ).add_to(m)

    # Marqueur prédiction
    folium.Marker(
        location=[pred_lat, pred_lon],
        popup=f"Bateau {mmsi} - Prediction +5min",
        icon=folium.Icon(color="blue", icon="hourglass-half", prefix='fa')
    ).add_to(m)

    # Marqueur position réelle correspondante
    if real_lat is not None and real_lon is not None:
        folium.Marker(
            location=[real_lat, real_lon],
            popup=f"Bateau {mmsi} - True +5min (approximated)",
            icon=folium.Icon(color="red", icon="map-marker", prefix='fa')
        ).add_to(m)

        # Calcul distance et ajout d'une ligne entre les 2 points
        dist_km = haversine(pred_lat, pred_lon, real_lat, real_lon)

        # Ligne entre la position prédite et réelle
        folium.PolyLine(
            locations=[(pred_lat, pred_lon), (real_lat, real_lon)],
            color="purple",
            weight=2,
            opacity=0.7,
            popup=f"Distance: {dist_km:.2f} km"
        ).add_to(m)

        # Popup distance sur la ligne (il faut contourner car PolyLine n'a pas de popup natif)
        midpoint = [(pred_lat + real_lat) / 2, (pred_lon + real_lon) / 2]
        folium.Marker(
            location=midpoint,
            icon=folium.DivIcon(
                html=f'<div style="font-size:12px; color:purple; font-weight:bold;">{dist_km:.2f} km</div>'
            )
        ).add_to(m)

# Sauvegarder et afficher la carte
m.save("map_10_bateaux_distance.html")
print("Carte sauvegardée : map_10_bateaux_distance.html")

display(m)